# Comparison

In [70]:
import time
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns

from common import *

%matplotlib inline
import visualization as viz
viz.set_plot_style()

print('numpy ver.: ' + np.__version__)
print('pandas ver.: ' + pd.__version__)

numpy ver.: 1.12.1
pandas ver.: 0.20.1


In [71]:
data = prep_data('../data/4A_1_201705_201710.csv')
print(len(data))

1213677


Links included in the evaluation, i.e. the first 32 links of 4A in the CCW direction:

In [72]:
data = data[(1 <= data['LineDirectionLinkOrder']) & (data['LineDirectionLinkOrder'] <= 32)]
assert len(data['LinkRef'].unique()) == 32
n = len(data)
print(n)

943136


In [73]:
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [109]:
mask = ('2017-10-12 06:00' <= data['DateTime'] ) & (data['DateTime'] < '2017-10-12 22:00') & (data['LinkRef'] == '1190:1192')

In [110]:
ts_sel = data[mask].set_index('DateTime')['LinkTravelTime']
len(ts_sel)

170

In [113]:
ts_5min = ts_sel.resample('5min').mean().interpolate()
ts_15min = ts_sel.resample('15min').mean().interpolate()
ts_30min = ts_sel.resample('30min').mean().interpolate()

In [120]:
tss = [ts_5min, ts_15min, ts_30min]
figs = ['5min', '15min', '30min']
cols = ['C1', 'C2', 'C3']

for i in range(len(tss)):
    fig, ax = plt.subplots(figsize = (9, 6))
    plt.plot(ts_sel, linestyle = '', marker = 'o', color = 'C0')
    plt.plot(tss[i], linestyle = '-', color = cols[i])

    dt_ix = pd.date_range(min(ts_15min.index), max(ts_15min.index) + pd.to_timedelta('00:15:00'), freq = '4H')
    ax.set_xticks(dt_ix[dt_ix.indexer_between_time('06:00', '00:00')])
    xfmt = md.DateFormatter('%H:%M')

    ax.xaxis.set_major_formatter(xfmt)
    #ax.set_xlabel('Time')
    #ax.set_ylabel('Link Travel Time (min)')
    
    plt.savefig('../paper/plots/decycle_' + figs[i] + '_n.pdf', bbox_inches='tight')
    plt.clf()